# Lithuania COVID analysis

I had a task to identify the 60-day interval of the pandemic, during which the total mortality was the largest in Lithuania. 


In [281]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import pylab as pl
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
%matplotlib inline
import matplotlib as mpl
from scipy import stats
import seaborn as sns
import locale
import datetime
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display
from sklearn import preprocessing

In [282]:
df = pd.read_csv('Agreguoti_COVID19_atvejai_ir_mirtys.csv')

In [283]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294692 entries, 0 to 294691
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   object_id          294692 non-null  int64 
 1   municipality_name  294692 non-null  object
 2   date               294692 non-null  object
 3   sex                294692 non-null  object
 4   age_gr             294692 non-null  object
 5   new_cases          294692 non-null  int64 
 6   deaths_all         294692 non-null  int64 
 7   deaths_cov1        294692 non-null  int64 
 8   deaths_cov2        294692 non-null  int64 
 9   deaths_cov3        294692 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 22.5+ MB


In [284]:
df['date'] = pd.to_datetime(df['date'])

In [285]:
df.sort_values('deaths_all',ascending=False)

,object_id,municipality_name,date,sex,age_gr,new_cases,deaths_all,deaths_cov1,deaths_cov2,deaths_cov3
4950,4951,Vilniaus m. sav.,2020-12-26,Moteris,80-89,12,11,1,6,7
12035,12036,Vilniaus m. sav.,2021-01-26,Vyras,70-79,7,10,2,3,4
5879,5880,Vilniaus m. sav.,2020-09-18,Moteris,80-89,0,10,0,0,0
15704,15705,Kauno m. sav.,2021-12-31,Moteris,80-89,3,9,2,3,3
4889,4890,Vilniaus m. sav.,2021-01-01,Moteris,80-89,8,9,3,4,4
...,...,...,...,...,...,...,...,...,...,...
144837,144838,Kupiškio r. sav.,2021-10-07,Moteris,90-99,1,0,0,0,0
144838,144839,Kupiškio r. sav.,2021-10-06,Moteris,0-9,1,0,0,0,0
144839,144840,Kupiškio r. sav.,2021-10-06,Moteris,10-19,2,0,0,0,0
144840,144841,Kupiškio r. sav.,2021-10-06,Moteris,40-49,1,0,0,0,0


In [287]:
df2=df.groupby('date',as_index=False)[['deaths_all']].sum() #creating new df

In [288]:
df2.head()

,date,deaths_all
0,2020-02-01,92
1,2020-02-02,102
2,2020-02-03,103
3,2020-02-04,116
4,2020-02-05,114


In [289]:
df2['60 dienu suma'] =df2['deaths_all'].rolling(60).sum() #rolling sum of cases 60 days before

In [290]:
df2.iloc[0:60,2].sum() #counting first 60 cases of df

6592.0

In [291]:
df2['60 dienu suma'].max() #finding max value

10615.0

In [247]:
df2.loc[df2['60 dienu suma']==10615,'date'] #finding the row index

352   2021-01-18
Name: date, dtype: datetime64[ns]

In [254]:
df2.iloc[292:352,0:2] #finding the 60 days period


,date,deaths_all
292,2020-11-19,134
293,2020-11-20,152
294,2020-11-21,177
295,2020-11-22,188
296,2020-11-23,151
297,2020-11-24,154
298,2020-11-25,169
299,2020-11-26,146
300,2020-11-27,171
301,2020-11-28,151


In [264]:
df3=df.loc[(df['date'] > '2020-11-19') & (df['date'] < '2021-01-19')]

In [273]:
df4=df3.groupby(['municipality_name','date'],as_index=False)[['deaths_all']].sum()  #creating new df for second task

In [275]:
df4

,municipality_name,date,deaths_all
0,Akmenės r. sav.,2020-11-20,4
1,Akmenės r. sav.,2020-11-21,5
2,Akmenės r. sav.,2020-11-22,2
3,Akmenės r. sav.,2020-11-23,3
4,Akmenės r. sav.,2020-11-24,1
...,...,...,...
3576,Švenčionių r. sav.,2021-01-14,0
3577,Švenčionių r. sav.,2021-01-15,3
3578,Švenčionių r. sav.,2021-01-16,3
3579,Švenčionių r. sav.,2021-01-17,1


In [278]:
df4.to_csv('df_4.csv') # saving file

task 2.1 was done with Microsof Power BI